In [1]:
import requests, json
import pyecharts
import pandas as pd
from pyecharts.charts import Pie
from pyecharts import options as opts

In [2]:
# 获取新冠肺炎最新的全球数据
resp = requests.get("https://lab.isaaclin.cn/nCoV/api/area?latest=1")
data_json=resp.json()
data_global=data_json.get("results")
df_global = pd.read_json(json.dumps(data_global),orient='records')

In [3]:
# 查看数据
df_global.head()

,cities,comment,confirmedCount,continentName,countryName,curedCount,currentConfirmedCount,deadCount,locationId,provinceName,provinceShortName,suspectedCount,updateTime
0,None,,2016,欧洲,瑞典,17,1974,25,962005,瑞典,瑞典,0,1585044504254
1,None,,875,欧洲,卢森堡,0,867,8,961004,卢森堡,卢森堡,0,1585044504254
2,None,,180,欧洲,拉脱维亚,0,180,0,964003,拉脱维亚,拉脱维亚,0,1585044504254
3,None,,115,欧洲,法罗群岛,0,115,0,0,法罗群岛,法罗群岛,0,1585044504254
4,None,,158,南美洲,乌拉圭,0,158,0,973011,乌拉圭,乌拉圭,0,1585044504254


In [4]:
# 只取关心的数据列
selected_cloumns = ['countryName','provinceShortName','provinceName','confirmedCount','currentConfirmedCount','deadCount','curedCount','updateTime']
# 过滤出确诊人数过千的海外国家的数据并按确诊人数倒序排列
df_selected_oversea=df_global[df_global.countryName!='中国'][selected_cloumns]
df_sorted_oversea=df_selected_oversea[df_selected_oversea.confirmedCount>1000].sort_values(by='confirmedCount', ascending=False)

In [5]:
# 查看过滤排序后的数据
df_sorted_oversea.head()

,countryName,provinceShortName,provinceName,confirmedCount,currentConfirmedCount,deadCount,curedCount,updateTime
189,意大利,意大利,意大利,64378,50869,6077,7432,1585040981468
22,美国,美国,美国,46450,45679,593,178,1585044504254
121,西班牙,西班牙,西班牙,33089,27552,2182,3355,1585040981468
98,德国,德国,德国,29056,28480,123,453,1585040981468
55,伊朗,伊朗,伊朗,24811,13964,1934,8913,1585044367433


In [6]:
# 定义绘制南丁格尔玫瑰图的函数
def plot_rose(item_list, data_list, title_string):
    ziped_item_data_list = [list(z) for z in zip(item_list, data_list)]

    rose_diagram = Pie(init_opts=opts.InitOpts(width='650px', height='650px'))
    rose_diagram.add("", ziped_item_data_list, radius=["40%", "130%"], center=["50%", "70%"], rosetype="area")
    rose_diagram.set_global_opts(title_opts=opts.TitleOpts(title=title_string),
                                 legend_opts=opts.LegendOpts(is_show=False),
                                 toolbox_opts=opts.ToolboxOpts()
                                )
    label_opts = opts.LabelOpts(is_show=True, position="inside", font_size=12, formatter="{b}:{c}人")
    rose_diagram.set_series_opts(label_opts=label_opts)
    rose_diagram.render('{}.html'.format(title_string))
    return rose_diagram

In [7]:
# 准备绘图使用的数据，仅取海外确诊过千国家的数据
item_oversea_list = df_sorted_oversea['countryName'].values.tolist()
data_oversea_list = df_sorted_oversea['confirmedCount'].values.tolist()
title_string = "rose-diagram"

In [8]:
# 绘制玫瑰图
rose2 = plot_rose(item_oversea_list, data_oversea_list, title_string)

In [9]:
# 将绘制的图渲染在juypter notebook中
rose2.render_notebook()